### 1. Aleatorização

O conjunto de códigos abaixo replica o Capítulo 4, intitulado **"Seleção Aleatória"**, do livro *Avaliação de Impacto na Prática*, segunda edição, dos autores Paul J. Gertler, Sebastián Martínez, Patrick Premand, Laura B. Rawlings e Christel M. J. Vermeersch. Este capítulo aborda a avaliação de impacto de políticas públicas utilizando o método de seleção aleatória, também conhecido como experimentos aleatórios controlados (randomized controlled trials, ou RCTs). O livro, em formato digital, pode ser acessado gratuitamente no seguinte site: [Avaliação de Impacto na Prática](https://publications.iadb.org/pt/avaliacao-de-impacto-na-pratica-segunda-edicao).


### 2. Bibliotecas

In [24]:
# Manipulação dos dados
import pandas as pd
import numpy as np

# Estatística
from scipy.stats import ttest_ind
import statsmodels.api as sm

# Configurações
import warnings
import os

# Exibição de informação
from tabulate import tabulate

### 3. Configurações

In [6]:
os.chdir("C:\\Users\\joaos\\Documents\\MeusProjetos\\modelos_avaliacao_impacto\\Dados") 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
warnings.filterwarnings("ignore")

### 4. Manipulando os dados

In [8]:
# Importando os dados
dados = pd.read_stata('evaluation.dta')

# Primeiras observações
dados.head()

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
0,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,0.0,15.185455,24.0,23.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
1,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,1.0,19.580902,25.0,24.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
2,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,0.0,13.076257,30.0,26.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,0.0
3,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,1.0,2.398854,31.0,27.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,1.0
4,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,1.0,0.000000,59.0,57.0,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,1.0


In [9]:
# Informações sobre o DataFrame
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19827 entries, 0 to 19826
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   locality_identifier   19827 non-null  float32
 1   household_identifier  19827 non-null  float32
 2   treatment_locality    19827 non-null  float32
 3   promotion_locality    19827 non-null  float32
 4   eligible              19827 non-null  float32
 5   enrolled              19827 non-null  float32
 6   enrolled_rp           19827 non-null  float32
 7   poverty_index         19827 non-null  float32
 8   round                 19827 non-null  float32
 9   health_expenditures   19827 non-null  float32
 10  age_hh                19827 non-null  float32
 11  age_sp                19827 non-null  float32
 12  educ_hh               19827 non-null  float32
 13  educ_sp               19827 non-null  float32
 14  female_hh             19827 non-null  float32
 15  indigenous         

In [10]:
# Estatísticas sobre das variáveis
dados.describe()

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
count,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,11257.000000
mean,73.933472,8038.973145,0.500277,0.512685,0.567761,0.299037,0.293287,56.789566,0.500025,17.035091,46.683117,40.581734,2.833550,2.618601,0.099057,0.352903,5.178645,0.603621,0.615978,2.079740,105.322517,0.052323
std,55.076504,4569.480957,0.500013,0.499852,0.495400,0.457847,0.455280,10.686109,0.500013,9.291581,15.294824,12.822698,2.754713,2.543279,0.298746,0.477885,2.195150,0.489157,0.486375,3.133202,42.063479,0.222687
min,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.479134,0.000000,0.000000,14.000000,14.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,9.465392,0.000000
25%,30.000000,3989.500000,0.000000,0.000000,0.000000,0.000000,0.000000,49.652241,0.000000,11.621977,34.000000,31.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,72.804218,0.000000
50%,59.000000,8040.000000,1.000000,1.000000,1.000000,0.000000,0.000000,56.414219,1.000000,16.051678,45.000000,41.000000,2.000000,2.000000,0.000000,0.000000,5.000000,1.000000,1.000000,1.000000,113.556497,0.000000
75%,112.000000,12033.000000,1.000000,1.000000,1.000000,1.000000,1.000000,62.905720,1.000000,21.236842,58.000000,48.000000,4.000000,4.000000,0.000000,1.000000,7.000000,1.000000,1.000000,3.000000,141.097735,0.000000
max,200.000000,15778.000000,1.000000,1.000000,1.000000,1.000000,1.000000,100.000000,1.000000,116.830643,88.000000,88.000000,16.000000,17.000000,1.000000,1.000000,13.000000,1.000000,1.000000,23.000000,170.457647,1.000000


In [ ]:
# Verificando a existência de dados faltantes. existem 8570 valores faltantes na coluna "hospital"
valores_faltantes = dados.isnull().sum().sort_values(ascending=False)
print(valores_faltantes)

hospital                8570
household_identifier       0
hospital_distance          0
land                       0
bathroom                   0
dirtfloor                  0
hhsize                     0
indigenous                 0
female_hh                  0
educ_sp                    0
educ_hh                    0
locality_identifier        0
age_hh                     0
health_expenditures        0
round                      0
poverty_index              0
enrolled_rp                0
enrolled                   0
eligible                   0
promotion_locality         0
treatment_locality         0
age_sp                     0
dtype: int64


In [17]:
# Filtrando para a linha de base e os elegíveis
dados = dados[dados['round'] == 0]
dados = dados[dados['eligible'] == 1]

Antes do início da replicação dos quadros, abaixo, calcula-se a média da despesa com despesa por saúde (em US$ per capita, por ano, para tratados e controle) 1.0 se tratado, 0.0 se controle. na linha de base, antes da intervenção.

In [20]:
dados.groupby('treatment_locality', as_index=False)['health_expenditures'].mean()

,treatment_locality,health_expenditures
0,0.0,14.573846
1,1.0,14.489694


### 5. Replicando os resultados do capítulo

In [21]:
tratado = dados[dados['treatment_locality'] == 1]
controle = dados[dados['treatment_locality'] == 0]

#### 5.1 - Quadro 4.1 - Avaliar o HISP: balanceamento entre os povoados de tratamento e de comparação na linha de base

Para validar o pressuposto de que nenhum outro fator, além da intervenção do programa, explica os resultados, é importante verificar se as famílias dos povoados de tratamento e de comparação apresentam características semelhantes na linha de base. Para isso, utilizamos a estatística t e o p-valor. Se o p-valor for maior que o nível de significância, como 5%, não rejeitamos a hipótese nula, ou seja, concluímos que as médias são estatisticamente iguais. Por outro lado, se o p-valor for menor que o nível de significância, rejeitamos a hipótese nula e concluímos que as médias são estatisticamente diferentes. No exemplo abaixo, verificamos que as médias de despesa com saúde entre os povoados tratados e não tratados são estatisticamente iguais.

In [25]:
# Variáveis analisadas
selected_columns = ['health_expenditures', 'age_hh', 'age_sp', 'educ_hh', 'educ_sp', 'female_hh', 'indigenous', 'hhsize',
                   'dirtfloor', 'bathroom', 'land', 'hospital_distance']

# Calcula a média de cada variável selecionada para tratados e não tratados
mean_by_locality = dados.groupby('treatment_locality')[selected_columns].mean()

# Executar teste t independente de duas amostras para cada variável selecionada
t_test_results = []
for column in selected_columns:
    result = ttest_ind(tratado[column], controle[column], equal_var=False)
    t_statistic = result.statistic
    p_value = result.pvalue
    t_test_results.append([column, mean_by_locality.loc[1, column], mean_by_locality.loc[0, column], 
                           mean_by_locality.loc[1, column] - mean_by_locality.loc[0, column], 
                           t_statistic, p_value])

# Imprimir os resultados
headers = ["Variable Name", "Treatment Locality=1", "Treatment Locality=0", "Difference", "T-Statistic", "P-Value"]
print(tabulate(t_test_results, headers=headers, floatfmt=".6f"))

Variable Name          Treatment Locality=1    Treatment Locality=0    Difference    T-Statistic    P-Value
-------------------  ----------------------  ----------------------  ------------  -------------  ---------
health_expenditures               14.489694               14.573846     -0.084152      -0.733920   0.463028
age_hh                            41.656578               42.292042     -0.635464      -1.690250   0.091037
age_sp                            36.836369               36.875000     -0.038631      -0.123787   0.901488
educ_hh                            2.971180                2.810383      0.160798       2.305089   0.021198
educ_sp                            2.703273                2.674362      0.028911       0.431490   0.666129
female_hh                          0.073212                0.077327     -0.004115      -0.583813   0.559370
indigenous                         0.429150                0.420045      0.009105       0.689916   0.490276
hhsize                      

Conforme observado acima, apenas as variáveis educ_hh (escolaridade do chefe de família) e hospital_distance (distância até o hospital) apresentaram diferenças estatisticamente significativas. Ambos os p-valores associados foram menores que 5%, o que nos levou a rejeitar a hipótese nula de igualdade das médias entre os grupos de tratamento e de comparação. No entanto, apesar de estatisticamente significativas, essas diferenças são pequenas. Como mostrado no Quadro 4.1, a diferença na escolaridade do chefe de família foi de apenas 0,16 ano, o que representa menos de 6% da média de anos de escolaridade do grupo de comparação. A diferença na distância até o hospital foi de 2,91 km, o que corresponde a menos de 3% da distância média até um hospital no grupo de comparação.

#### 5.2 - Quadro 4.2 - Avaliar o HISP: seleção aleatória com comparação de médias

In [29]:
# importando novamente os dados
dados = pd.read_stata('evaluation.dta')

# Filtrando para o acompanhemento e os elegíveis
dados = dados[dados['round'] == 1]
dados = dados[dados['eligible'] == 1]

# Separando entre tratados e controle
tratado = dados[dados["treatment_locality"]==1]
controle = dados[dados["treatment_locality"]==0]

In [30]:
# Calcule a média de 'health_expenditures' para cada 'treatment_locality'.
mean_by_locality = dados.groupby('treatment_locality')['health_expenditures'].mean()

# Realiza um teste t de duas amostras independentes.
result = ttest_ind(tratado['health_expenditures'], controle['health_expenditures'], equal_var=False)

# Extrai a estatística do teste e o p-valor do resultado
t_statistic = result.statistic
p_value = result.pvalue

# Calcula a diferença entre as médias de treatment_locality=1 e treatment_locality=0
mean_diff = mean_by_locality[1] - mean_by_locality[0]

# Prepara os dados para o tabulate
table_data = [
    ['Treatment Locality=1', 'Treatment Locality=0', 'Difference', 'T-Statistic', 'P-Value'],
    [mean_by_locality[1], mean_by_locality[0], mean_diff, t_statistic, p_value]
]

# Imprime os resultados usando o tabulate
print(tabulate(table_data, headers="firstrow", floatfmt=".6f"))

  Treatment Locality=1    Treatment Locality=0    Difference    T-Statistic    P-Value
----------------------  ----------------------  ------------  -------------  ---------
              7.840179               17.980551    -10.140371     -49.346764   0.000000


Como observado no Quadro 4.1, no início do estudo, ou seja, na linha de base, os gastos médios com saúde entre os grupos de tratamento e de comparação não apresentavam diferenças estatísticas significativas. No entanto, como mostrado no Quadro 2, o impacto do programa resultou em uma redução de US$ 10,14 nos gastos com saúde ao longo de dois anos, como é exibido na coluna "Difference".

#### 5.3 - Quadro 4.3 - Avaliar o HISP: seleção aleatória com análise de regressão

Replicar esse resultado por meio de uma análise de regressão linear produzirá o mesmo número, conforme mostra o quadro 4.3. Para avaliar o impacto que povoados tratados sofrerão temos que avaliar no período  t + 1, neste caso, round = 1, a variável que determina o impacto é treatment_locality


In [35]:
dados = pd.read_stata('evaluation.dta')

dados = dados[dados["round"] == 1]
dados = dados[dados["eligible"] == 1]

tratado = dados[dados["treatment_locality"]==1]
controle = dados[dados["treatment_locality"]==0]

In [ ]:
# Regressão linear simples

# Variável dependente
y = dados['health_expenditures']

# Variável independente
x = dados['treatment_locality']

x = sm.add_constant(x)
modelo = sm.OLS(y, x).fit()
previsao = modelo.predict(x)
imprimir = modelo.summary()
print(imprimir)

                             OLS Regression Results                            
Dep. Variable:     health_expenditures   R-squared:                       0.300
Model:                             OLS   Adj. R-squared:                  0.300
Method:                  Least Squares   F-statistic:                     2416.
Date:                 Tue, 05 Nov 2024   Prob (F-statistic):               0.00
Time:                         15:13:20   Log-Likelihood:                -19497.
No. Observations:                 5629   AIC:                         3.900e+04
Df Residuals:                     5627   BIC:                         3.901e+04
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 17.9

Assim como no Quadro 4.2, que apresenta a comparação de médias, a análise de regressão linear também indica que o impacto do programa resulta em uma redução de US$ 10,14, conforme mostrado pelo coeficiente da variável treatment_locality, que é -10.1404.

#### 5.4 - Regressão linear multivariada - Avaliar o HISP: seleção aleatória com análise de regressão multivarada

Por fim, como sugere o livro será realizada uma regressão multivariada com variáveis selecionadas.

In [38]:
# Importando os dados
dados = pd.read_stata('evaluation.dta')

dados = dados[dados["round"] == 1]
dados = dados[dados["eligible"] == 1]

tratado = dados[dados["treatment_locality"]==1]
controle = dados[dados["treatment_locality"]==0]

# Selecionando a variável dependente (y) e as variáveis independentes (X)
y = dados['health_expenditures']
X = dados[['treatment_locality', 'age_hh', 'age_sp', 'educ_hh', 'educ_sp', 
           'female_hh', 'indigenous','hhsize', 'dirtfloor', 'bathroom', 'land', 
           'hospital_distance']]

In [39]:
# Realizando a regressão multivariada
X = sm.add_constant(X)
modelo = sm.OLS(y, X).fit()
previsao = modelo.predict(X)

In [40]:
# Imprimindo os resultados da regressão
imprimir = modelo.summary()
print(imprimir)

                             OLS Regression Results                            
Dep. Variable:     health_expenditures   R-squared:                       0.430
Model:                             OLS   Adj. R-squared:                  0.428
Method:                  Least Squares   F-statistic:                     352.6
Date:                 Tue, 05 Nov 2024   Prob (F-statistic):               0.00
Time:                         15:30:08   Log-Likelihood:                -18922.
No. Observations:                 5629   AIC:                         3.787e+04
Df Residuals:                     5616   BIC:                         3.796e+04
Df Model:                           12                                         
Covariance Type:             nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 27.5

Ao contrário da comparação de médias apresentada no Quadro 2 e da regressão linear univariada, a análise de regressão multivariada revela que, após dois anos, o efeito do programa é uma redução de US 10,01, conforme indicado pelo coeficiente da variável treatment_locality. Isso significa que a participação no programa está associada a uma diminuição de US 10,01 nos gastos per capita com saúde. Além disso, esse coeficiente é estatisticamente significativo, pois seu p-valor é inferior a 5%.  